In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime

path="./rawdata/data3.csv"
file=open(path, newline='')
reader=csv.reader(file)


In [2]:
header=next(reader)
#separate data, label for training and test 
training_set=[]
test_set=[]
training_label=[]
test_label=[]
label_count=np.zeros(16)

for row in reader:
    msec= []
    
    #for acc
    X_acc=[]
    Y_acc=[]
    Z_acc=[]
    #for gyro
    X_gyro=[]
    Y_gyro=[]
    Z_gyro=[]
    
    #row = [Date, msec,label,X_acc,Y_acc,Z_acc,X_gyro,Y_gyro,Z_gyro]
    date=datetime.strptime(row[0],'%Y/%m/%d')
    msec=row[1:3]
    labl=int(row[3]) #label
    label_count[labl]+=1
    row[4:]=[float(i) for i in row[4:]]
        
    X_acc=(row[4:604])
    Y_acc=(row[604:1204])
    Z_acc=(row[1204:1804])
    X_gyro=(row[1804:2404])
    Y_gyro=(row[2404:3004])
    Z_gyro=(row[3004:3604])

    #6*6*100 float array : will be feature-extracted
    window=np.array([[X_acc[0:100],X_acc[100:200],X_acc[200:300],X_acc[300:400],X_acc[400:500],X_acc[500:600]],
                [Y_acc[0:100],Y_acc[100:200],Y_acc[200:300],Y_acc[300:400],Y_acc[400:500],Y_acc[500:600]],
                [Z_acc[0:100],Z_acc[100:200],Z_acc[200:300],Z_acc[300:400],Z_acc[400:500],Z_acc[500:600]],
                [X_gyro[0:100],X_gyro[100:200],X_gyro[200:300],X_gyro[300:400],X_gyro[400:500],X_gyro[500:600]],
                [Y_gyro[0:100],Y_gyro[100:200],Y_gyro[200:300],Y_gyro[300:400],Y_gyro[400:500],Y_gyro[500:600]],
                [Z_gyro[0:100],Z_gyro[100:200],Z_gyro[200:300],Z_gyro[300:400],Z_gyro[400:500],Z_gyro[500:600]]])
    
    #5 features  what else?2
    window_mean=window.mean(axis=-1)
    window_stddev=window.std(axis=-1)
    window_median=np.median(window,axis=-1)
    window_percent25=np.percentile(window,25,axis=-1)
    window_percent75=np.percentile(window,75,axis=-1)

    window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
    print(window_feature.shape)
    window_feature=window_feature.reshape(180)
    
    if (label_count[labl]%5!=1):
        training_set.append(window_feature)
        training_label.append(labl)
    else:
        test_set.append(window_feature)
        test_label.append(labl)

#label=int(row[3])
#print(window_feature.shape)
#parsing complete

(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)
(5, 1, 6, 6)

In [8]:
print(window_feature.shape)
print(training_label)
print(test_set)

(180,)
[7, 7, 1, 1, 7, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 6, 6, 0, 5, 5, 5, 2, 2, 7, 0, 6, 6, 5, 3, 12, 6, 6, 12, 12, 12, 9, 7, 0, 3, 1, 1, 12, 7, 0, 12, 12, 12, 6, 6, 6, 12, 12, 12, 12, 12, 12, 12, 0, 0, 7, 0, 12, 12, 12, 9, 12, 6, 12, 0, 12, 12, 5, 5, 2, 2, 12, 12, 6, 6, 12, 12, 12, 12, 6, 3, 6, 7, 0, 0, 5, 2, 5, 5, 5, 5, 0, 7, 0, 0, 3, 3, 12, 12, 0, 12, 1, 1, 1, 5, 6, 6, 6, 6, 1, 7, 0, 1, 6, 2, 1, 1, 1, 1, 5, 6, 10, 10, 10, 10, 1, 9, 9, 5, 13, 13, 13, 13, 13, 13, 13, 13, 8, 8, 8, 8, 14, 14, 14, 14, 8, 0, 14, 8, 8]
[array([-7.29045170e-01, -1.96931692e+00, -2.92881500e-02, -3.05415099e+00,
       -1.91832596e+00,  4.74066020e-01,  8.47047249e+00,  4.94290158e+00,
        3.66123395e+00,  1.81919126e+00,  6.75084658e+00,  8.35480333e+00,
        3.40340733e+00, -7.80344924e+00, -9.01314080e+00, -8.80836296e+00,
       -3.03385890e-01,  5.13468629e+00,  5.85830001e+01,  2.27339000e+01,
        2.04050002e+01, -1.09347002e+01, -6.72427003e+01,  3.37260010e+00,
        6.17609998e+01,  2.1356

In [11]:
#SVM
#prepare the data
X=training_set
y=training_label
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)

clf.fit(X,y)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
test_pred1=clf.predict(test_set)

ohc=OneHotEncoder(categories=[range(16)])
onehot_pred1=test_pred1.reshape(45,1)
print(test_pred1.shape)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
#print(test_pred1)
#print(onehot_pred1)

#print(accuracy_score(test_pred1,test_label))
#print(clf.n_support_)
#confusion_matrix(test_pred1, test_label)

(45,)


In [12]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
clf.fit(X, y)
test_pred2=clf.predict(test_set)
print(accuracy_score(test_pred2,test_label))

onehot_pred2=test_pred2.reshape(45,-1)
onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

0.5111111111111111


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier()
clf.fit(X, y)
test_pred3=clf.predict(test_set)
#print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
onehot_pred3=test_pred2.reshape(45,-1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
print(onehot_pred3)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [13]:
#ensemble 1 : unanimous consensus
ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
print(ensemble1_pred)

[ 0  7  1  6  6  0 12  6  5  7 12 12 12 12  1  0  6  0 12  0 12  0  0  5
  0  0 12  0  0  0  5  0  1  1  0  0  0  0  0 13  8  0  8  0 13]


In [14]:
#ensemble 2 : majority consensus
ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
ensemble2_pred= ensemble2_pred.astype('int64') 
ensemble2_pred= ensemble2_pred.astype('float64') 
ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
print(ensemble2_pred)
print(test_label)
print(accuracy_score(ensemble2_pred,test_label))


[ 2  7  1  6  6  0 12  6  5  7 12 12 12 12  1 14  6 13 12 13 12  3  5  5
  0 14 12  9 10  7  5  7  1  1 14  5 14  9 14 13  8 14  8 14 13]
[6, 7, 1, 6, 2, 0, 0, 6, 5, 7, 3, 4, 12, 9, 1, 12, 6, 12, 12, 0, 12, 6, 12, 5, 0, 12, 12, 6, 7, 2, 5, 0, 3, 1, 5, 6, 1, 10, 13, 13, 8, 14, 8, 14, 13]
0.5111111111111111
